### Opportunity to explore vendor payments(City of Scottsdale) and experiment with pandas styling features.  Following along from example by [Chris Moffitt](https://pbpython.com/styling-pandas.html)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker
import seaborn as sns
import datetime

vendor_payments = pd.read_csv("data/Scottsdale/ct_VendorPayments.csv")

vendor_payments['PaymentDate'] = pd.to_datetime(vendor_payments['PaymentDate'])
vendor_payments['Month'] = vendor_payments['PaymentDate'].dt.month
vendor_payments['Day_of_Year'] = vendor_payments['PaymentDate'].dt.dayofyear

In [3]:
# date range of data
print(vendor_payments['PaymentDate'].min())
print(vendor_payments['PaymentDate'].max())

2017-08-14 00:00:00
2018-08-10 00:00:00


In [4]:
vendor_payments.head()

,PaymentDate,VendorName,PaymentAmount,Fund,FundDescription,Center,CenterName,Account,AccountName,FiscalYear,AccountingPeriod,Month,Day_of_Year
0,2017-08-14,A TO Z EQUIPMENT RENTAL & SALES,1021.38,200,TRANSPORTATION,20860,TRAFFIC SIGNALS,53790,"MAT TO MAINT & REPAIR MACH, EQUIP",2018,2,8,226
1,2017-08-14,AIMS PVIC,380.00,100,GENERAL,25411,IRRIGATION REPAIR,52360,"MAINT - MACHINERY, EQUIP & AUTOS",2018,2,8,226
2,2017-08-14,AIMS PVIC,2125.00,200,TRANSPORTATION,20810,GRADING AND DRAINAGE,52390,MISC MAINTENANCE AND REPAIR,2018,2,8,226
3,2017-08-14,AIMS PVIC,15658.03,603,WATER RECLAMATION,29321,SEWER SYSTEM,52190,OTHER PROFESSIONAL SERVICES,2018,2,8,226
4,2017-08-14,ALL CITY TOWING,583.00,700,FLEET MANAGEMENT,20706,FLEET MANAGEMENT OPERATIONS,52360,"MAINT - MACHINERY, EQUIP & AUTOS",2018,2,8,226


In [5]:
# group by month and subtotal data for each month
monthly_payments = vendor_payments.groupby([pd.Grouper(key='PaymentDate', freq='M')]) \
    ['PaymentAmount'].agg('sum') \
    .reset_index()

# compute percent of total for each month
monthly_payments['percent_total'] = monthly_payments['PaymentAmount'] / monthly_payments['PaymentAmount'].sum()

# format data to be more readable
monthly_payments = monthly_payments.style.format({'PaymentAmount': '${0:,.0f}', 'percent_total': '{:.2%}'}).hide_index()
monthly_payments

PaymentDate,PaymentAmount,percent_total
2017-08-31 00:00:00,"$16,407,638",5.65%
2017-09-30 00:00:00,"$18,977,599",6.53%
2017-10-31 00:00:00,"$27,377,901",9.42%
2017-11-30 00:00:00,"$21,451,298",7.38%
2017-12-31 00:00:00,"$25,749,321",8.86%
2018-01-31 00:00:00,"$24,807,941",8.54%
2018-02-28 00:00:00,"$22,644,036",7.79%
2018-03-31 00:00:00,"$21,600,046",7.43%
2018-04-30 00:00:00,"$24,666,970",8.49%
2018-05-31 00:00:00,"$26,209,036",9.02%


In [8]:
monthly_payments \
    .bar(color='orange',subset=['PaymentAmount'], align='zero') \
    .bar(color='lightblue', subset=['percent_total'], align='zero') \
    .set_caption('Monthly Vendor Payments')

PaymentDate,PaymentAmount,percent_total
2017-08-31 00:00:00,"$16,407,638",5.65%
2017-09-30 00:00:00,"$18,977,599",6.53%
2017-10-31 00:00:00,"$27,377,901",9.42%
2017-11-30 00:00:00,"$21,451,298",7.38%
2017-12-31 00:00:00,"$25,749,321",8.86%
2018-01-31 00:00:00,"$24,807,941",8.54%
2018-02-28 00:00:00,"$22,644,036",7.79%
2018-03-31 00:00:00,"$21,600,046",7.43%
2018-04-30 00:00:00,"$24,666,970",8.49%
2018-05-31 00:00:00,"$26,209,036",9.02%


#### This is an easy way to glance at the data and get a sense for quantities.  However, I find myself usually sorting to get the largest/smallest values at the top.

In [9]:
# group by month and subtotal data for each month
fund_payments = vendor_payments.groupby('Fund') \
    ['PaymentAmount'].sum() \
    .reset_index()

# compute percent of total for each fund
fund_payments['percent_total'] = fund_payments['PaymentAmount'] / fund_payments['PaymentAmount'].sum()

# format data to be more readable
fund_payments = fund_payments \
    .style.format({'PaymentAmount': '${0:,.0f}', 'percent_total': '{:.2%}'}) \
    .hide_index() \
    .bar(color='skyblue', subset=['PaymentAmount'])

fund_payments

Fund,PaymentAmount,percent_total
100,"$44,507,853",15.31%
200,"$14,885,858",5.12%
220,"$707,034",0.24%
221,"$341,277",0.12%
225,"$184,820",0.06%
230,"$2,022,692",0.70%
266,"$3,000",0.00%
280,"$3,284,093",1.13%
285,"$14,175,547",4.88%
300,"$153,194",0.05%


In [10]:
vendor_payments.head()

,PaymentDate,VendorName,PaymentAmount,Fund,FundDescription,Center,CenterName,Account,AccountName,FiscalYear,AccountingPeriod,Month,Day_of_Year
0,2017-08-14,A TO Z EQUIPMENT RENTAL & SALES,1021.38,200,TRANSPORTATION,20860,TRAFFIC SIGNALS,53790,"MAT TO MAINT & REPAIR MACH, EQUIP",2018,2,8,226
1,2017-08-14,AIMS PVIC,380.00,100,GENERAL,25411,IRRIGATION REPAIR,52360,"MAINT - MACHINERY, EQUIP & AUTOS",2018,2,8,226
2,2017-08-14,AIMS PVIC,2125.00,200,TRANSPORTATION,20810,GRADING AND DRAINAGE,52390,MISC MAINTENANCE AND REPAIR,2018,2,8,226
3,2017-08-14,AIMS PVIC,15658.03,603,WATER RECLAMATION,29321,SEWER SYSTEM,52190,OTHER PROFESSIONAL SERVICES,2018,2,8,226
4,2017-08-14,ALL CITY TOWING,583.00,700,FLEET MANAGEMENT,20706,FLEET MANAGEMENT OPERATIONS,52360,"MAINT - MACHINERY, EQUIP & AUTOS",2018,2,8,226


### For each vendor, what funds are they being paid from? and in which months are they receiving those payments?


In [11]:
pd.pivot_table(vendor_payments, \
               index=['VendorName', 'Fund'], \
               columns=['Month'], \
               aggfunc=[np.sum], \
               values=['PaymentAmount']) \
               .reset_index() \
               .head()

VendorName Fund           sum  \
                                                          PaymentAmount   
Month                                                                 1   
0      1 800 RADIATOR                                 700           NaN   
1      1800 RADIATOR                                  700           NaN   
2      232 RANCH LLC                                  100           NaN   
3      3M                                             200       2444.75   
4      4IMPRINT INC                                   100           NaN   

                                                                            \
                                                                             
Month       2        3   4       5        6   7        8       9        10   
0         NaN      NaN NaN   96.16    96.16 NaN    96.16  852.35     96.16   
1         NaN      NaN NaN  556.03      NaN NaN      NaN     NaN       NaN   
2      7500.0      NaN NaN     NaN      NaN NaN      NaN     NaN       NaN   
3         NaN  2850.53 NaN     NaN      NaN NaN  1559.98     NaN  10909.73   
4         NaN      NaN NaN     NaN  3894.32 NaN   939.54     NaN       NaN   

                   
                   
Month      11  12  
0         NaN NaN  
1         NaN NaN  
2         NaN NaN  
3         NaN NaN  
4      613.74 NaN